In [197]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')
df_train.tail()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y
613,LP002990,Female,No,0,Graduate,Yes,4583,0.0,133.0,360.0,0.0,Semiurban,N


In [198]:
df_test = pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')
df_test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [199]:
y = df_train.loc[:,['Loan_Status']].values.ravel()

In [200]:
df_train = df_train.drop('Loan_Status', axis =1)
df= df_train.append(df_test)
df = df.drop('Loan_ID', axis=1).reset_index(drop=True)
df.tail()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
976,Male,Yes,3+,Not Graduate,Yes,4009,1777.0,113.0,360.0,1.0,Urban
977,Male,Yes,0,Graduate,No,4158,709.0,115.0,360.0,1.0,Urban
978,Male,No,0,Graduate,No,3250,1993.0,126.0,360.0,NaN,Semiurban
979,Male,Yes,0,Graduate,No,5000,2393.0,158.0,360.0,1.0,Rural
980,Male,No,0,Graduate,Yes,9200,0.0,98.0,180.0,1.0,Rural


In [201]:
df.isnull().sum()

Gender               24
Married               3
Dependents           25
Education             0
Self_Employed        55
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           27
Loan_Amount_Term     20
Credit_History       79
Property_Area         0
dtype: int64

In [202]:
df.Gender= df.Gender.fillna('Male')
df.Married= df.Married.fillna('Yes')
df.Dependents= df.Dependents.fillna(0)
df.Self_Employed= df.Self_Employed.fillna('No')
df.LoanAmount= df.LoanAmount.fillna(df.LoanAmount.mean())
df.Loan_Amount_Term= df.Loan_Amount_Term.fillna(360)
df.Credit_History= df.Credit_History.fillna(1)
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [203]:
df.Credit_History = df.Credit_History.astype(str)
df['Total_Income']= df['ApplicantIncome']+df['CoapplicantIncome']
df = pd.get_dummies(df, drop_first=True)
df.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Total_Income,Gender_Male,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Not Graduate,Self_Employed_Yes,Credit_History_1.0,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,142.51153,360.0,5849.0,1,0,1,0,0,0,0,0,1,0,1
1,4583,1508.0,128.00000,360.0,6091.0,1,1,0,1,0,0,0,0,1,0,0
2,3000,0.0,66.00000,360.0,3000.0,1,1,1,0,0,0,0,1,1,0,1
3,2583,2358.0,120.00000,360.0,4941.0,1,1,1,0,0,0,1,0,1,0,1
4,6000,0.0,141.00000,360.0,6000.0,1,0,1,0,0,0,0,0,1,0,1


In [204]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 11, random_state=42, whiten=True)
pca.fit(df)
df_pca = pca.transform(df)
X = df_pca[:614,:]
val = df_pca[614:, :]

In [205]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
max_features_rf = np.arange(3,9,3)
n_estimators_rf = [15,20,25]
max_depth_rf = np.arange(1,6, 2)
param_grid_rf = {'max_features':max_features_rf, 'n_estimators':n_estimators_rf, 'max_depth':max_depth_rf} 
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_cv = GridSearchCV(rf, param_grid_rf, cv=5)
rf_cv.fit(X,y)
print(rf_cv.best_score_)
print(rf_cv.best_params_)

0.8061889250814332
{'max_depth': 1, 'max_features': 6, 'n_estimators': 15}


In [206]:
output = rf_cv.predict(val)
df_output = pd.DataFrame()
df_output['Loan_ID'] = df_test['Loan_ID']
df_output['Loan_Status'] = output
df_output.to_csv('output.csv',index=False)